# Programación Lineal

Formule los problemas de programación lineal para cada uno de los ejercicios propuestos, solucione usando ORtools en Python y describa su respectiva solución, si existe.

## Librerias Necesarias

In [1]:
from ortools.linear_solver import pywraplp

## Primer Punto

### Proposición

Envigado, Palmira, Ibagué, las cuales pueden producir respectivamente 180, 110 y 230 carrocerías cada  una, para el año 2025 los sistemas masivos de transporte están solicitando están carrocerías de buses  BRT así: Bogotá 210, Cali 75, Bucaramanga 65 y Medellín 90, las carrocerías producidas en Envigado e  Ibagué pueden ser enviados a los almacenes de ensamble primario ubicados en Pereira y Armenia, pero  Palmira solo envía al almacén de ensamble primario ubicado en Armenia, estos almacenes de ensamble  primario, envían a su vez a cualquiera de los almacenes de terminado ubicados en Duitama y Cartago,  Ninguno de los almacenes ni de ensamble o terminado almacena carrocerías en inventario, por  consiguiente deben enviar todas las carrocerías que reciben.

Los clientes de Cali y Bucaramanga pueden recibir las carrocerías de cualquiera de los almacenes de terminado, sin embargo, por un tema de contratación los clientes de Bogotá deben obtener las carrocerías exclusivamente de Duitama y los de Medellín solo de Cartago, los costos de envío de las carrocerías a los almacenes de ensamble y de estos a los almacenes de terminado y de estos últimos a  los clientes se dan a continuación:

#### COSTOS DE ALMACENES DE ENSAMBLE TRANSPORTE

| FABRICAS | PEREIRA | ARMENIA |
| -------- | ------- | ------- |
| ENVIGADO | 500     | 1200    |
| PALMIRA  | -       | 2000    |
| IBAGUE   | 1300    | 2100    |

#### COSTOS DE TRANSPORTE ALMACENES DE TERMINADO


| ALM. ENSAMBLE | DUITAMA | CARTAGO |
| ------------- | ------- | ------- |
| PEREIRA       | 1900    | 1300    |
| ARMENIA       | 1450    | 1370    |

#### COSTOS DE TRANSPORTE CLIENTES FINALES

| ALM.TERMINADO | BOGOTA | CALI | B/MGA | MEDELLIN |
| ------------- | ------ | ---- | ----- | -------- |
| DUITAMA       | 2300   | 2100 | 1700  | -        |
| CARTAGO       | -      | 1400 | 2300  | 1800     |

![image.png](./images/Grafico%20punto%201.png)


### Model 

In [ ]:
solver = pywraplp.Solver.CreateSolver("GLOP")
infinity = solver.infinity()

#### Conjuntos:

- **Fábricas:** $\rightarrow i$ = {Envigado, Palmira, Ibagué}
- **Almacenes de Ensamble Primario:** $\rightarrow j$ = {Pereira, Armenia}
- **Almacenes de Terminado:** $\rightarrow k$ = {Duitama, Cartago}
- **Clientes:** $\rightarrow m$ = {Bogotá, Cali, Bucaramanga, Medellín}

#### Variables de Desición

- $x_{ij} \rightarrow$ de la fabrica $i$ al Almacén de emsamble $j$ $\forall{ij}$

- $y_{jk} \rightarrow$ del Almacén de Ensamble $j$ al Almacén de Terminado $k$ $\forall{jk}$  

- $P_{km} \rightarrow$ del Almacén de Terminado $k$ al Destino $m$ $\forall{km}$

In [ ]:
xEP = solver.Var(0, infinity, integer = False, name = "Envigado - Pereira") # 500
xEA = solver.Var(0, infinity, integer = False, name = "Envigado - Armenia") # 1200
xPA = solver.Var(0, infinity, integer = False, name = "Palmira - Armenia") # 2000
xIP = solver.Var(0, infinity, integer = False, name = "Ibague - Pereira") # 1300
xIA = solver.Var(0, infinity, integer = False, name = "Ibague - Armenia") # 2100

yPD = solver.Var(0, infinity, integer = False, name = "Pereira - Duitama") # 1900
yPC = solver.Var(0, infinity, integer = False, name = "Pereira - Cartago") # 1300
yAD = solver.Var(0, infinity, integer = False, name = "Armenia - Duitama") # 1450
yAC = solver.Var(0, infinity, integer = False, name = "Armenia - Cartago") # 1370

pDBo = solver.Var(0, infinity, integer = False, name = "Duitama - Bogotá") # 2300
pDC = solver.Var(0, infinity, integer = False, name = "Duitama - Cali") # 2100
pDBu = solver.Var(0, infinity, integer = False, name = "Duitama - Bucaramanga") # 1700
pCC = solver.Var(0, infinity, integer = False, name = "Cartago - Cali") # 1400
pCBu = solver.Var(0, infinity, integer = False, name = "Cartago - Bucaramanga") # 2300
pCM = solver.Var(0, infinity, integer = False, name = "Cartago - Medallin") # 1800


print("Numero de variables =", solver.NumVariables())

#### Restriciones

##### Oferta

In [ ]:
solver.Add(xEP + xEA <= 180)
solver.Add(xPA <= 110)
solver.Add(xIP + xIA <= 230)

##### Demanda

In [ ]:
solver.Add(pDBo >= 210)
solver.Add(pCM <= 90)
solver.Add(pDC + pCC <= 75)
solver.Add(pDBu + pCBu <= 65)

##### Entrada == Salida

In [ ]:
solver.Add(xEP + xIP - yPD - yPC == 0)
solver.Add(xEA + xPA + xIA - yAD - yAC == 0)

solver.Add(yPD + yAD - pDBo - pDC - pDBu == 0)
solver.Add(yPC + yAC - pCC - pCBu - pCM == 0)

print("Numero de restricciones", solver.NumConstraints())

#### Funcion Objetivo

$Min\;z = \sum_{ij}X_{ij}*C_{ij}+\sum_{jk}Y_{jk}*C_{jk}+\sum_{km}P_{km}*C_{km}$

In [ ]:

solver.Minimize(500*xEP + 1200*xEA + 2000*xPA 
				+ 1300*xIP + 2100*xIA 
				+ 1900*yPD + 1300*yPC + 1450*yAD + 1370*yAC 
				+ 2300*pDBo + 2100*pDC + 1700*pDBu + 1400*pCC + 2300*pCBu + 1800*pCM)

### Solucion

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("Solución")
    print("Valor de la Función Objetivo a Maximizar es: ", solver.Objective().Value())

    print("Cnt ", xEP, ": ", xEP.solution_value())
    print("Cnt ", xEA, ": ", xEA.solution_value())
    print("Cnt ", xPA, ": ", xPA.solution_value())
    print("Cnt ", xIP, ": ", xIP.solution_value())
    print("Cnt ", xIA, ": ", xIA.solution_value())
    print("Cnt ", yPD, ": ", yPD.solution_value())
    print("Cnt ", yPC, ": ", yPC.solution_value())
    print("Cnt ", yAD, ": ", yAD.solution_value())
    print("Cnt ", yAC, ": ", yAC.solution_value())
    print("Cnt", pDBo, ": ", pDBo.solution_value())
    print("Cnt ", pDC, ": ", pDC.solution_value())
    print("Cnt", pDBu, ": ", pDBu.solution_value())
    print("Cnt ", pCC, ": ", pCC.solution_value())
    print("Cnt", pCBu, ": ", pCBu.solution_value())
    print("Cnt ", pCM, ": ", pCM.solution_value())

else:
    print("El modelo no tiene una solución optima")

## Segundo Punto

### Proposición

### Model 

### Solucion

## Tercer Punto

### Proposición

La empresa francesa de trenes SNCF en su modelo de negocio para el transporte de pasajeros, considera en el trayecto París – Lyon las siguientes políticas para la venta de sus asientos, para no entrar en perdidas los asientos se deben vender como mínimo a 50 euros los de clase normal, y 75 euros los de clase Premium. Los pasajeros que compren los tiquetes con un tiempo mayor a 6 meses de anticipación pagaran el precio base más un 20% de incremento, los tiquetes con un tiempo mayor a 3 meses de anticipación pagaran el precio base más un 45% de incremento, el resto de los tiquetes vendidos se cobran teniendo en cuenta el precio base más un 80% de incremento. Los estudiantes obtienen un descuento de 10 euros en sus asientos clase normal sin importar la fecha de compra, y 8 para la clase premium solo si son comprados con un tiempo de anticipación mayor a 3 meses.

Los poseedores de la tarjeta joven, con menos de 26 años tienen un descuento de 8 euros en sus asientos clase normal sin importar la fecha de compra, y 4 para la clase premium solo si son comprados con un tiempo de anticipación mayor a 3 meses. Los adultos mayores cuentan con las mismas condiciones de los menores 26 años, solo que para ellos el descuento corresponde a la mitad del de los estudiantes.
El tren cuenta con 640 plazas disponibles, de las cuales el 35% son de clase Premium. La empresa estima que los estudiantes ocupan como mínimo la una cuarta parte del tren, las plazas Premium se venden al menos en un 40%, de estas los estudiantes adquieren como mínimo un 80% antes de 3 meses. Los adultos mayores ocupan como mínimo el 60% de la clase normal, al menos la mitad de estas son adquiridas antes del último mes

### Model 

In [3]:
solver = pywraplp.Solver.CreateSolver("GLOP")
infinity = solver.infinity()

model_data = {}

#### Conjunto(s)

- $i$ = {Normal, Premium}
- $j$ = {6MesesAntes, 3MesesAntes , normal}
- $k$ = {personas, personasViejietas, estudiantes, jovenes}

#### Variables de Desición

- $x_{ijk} \rightarrow$ Cantidad de asientos de $C_i$ comprados en el $tiempo_j$ siendo $persona_k$

In [4]:
model_data["num_variables"] = 24
model_data["Nombres variables"] = [
    "Normal-6antes-persona",
    "Normal-6antes-personaVieja",
    "Normal-6antes-estudiante",
    "Normal-6antes-joven",
    "Premium-6antes-persona",
    "Premium-6antes-personaVieja",
    "Premium-6antes-estudiante",
    "Premium-6antes-joven",
    # -
    "Normal-3antes-persona",
    "Normal-3antes-personaVieja",
    "Normal-3antes-estudiante",
    "Normal-3antes-joven",
    "Premium-3antes-persona",
    "Premium-3antes-personaVieja",
    "Premium-3antes-estudiante",
    "Premium-3antes-joven",
    # -
    "Normal-normal-persona",
    "Normal-normal-personaVieja",
    "Normal-normal-estudiante",
    "Normal-normal-joven",
    "Premium-normal-persona",
    "Premium-normal-personaVieja",
    "Premium-normal-estudiante",
    "Premium-normal-joven",
]

In [5]:
infinito = solver.infinity()
x = {}

for j in range(model_data["num_variables"]):
    x[j] = solver.Var(
        0, infinito, integer=False, name=model_data["Nombres variables"][j]
    )

print("numero de variables =", solver.NumVariables())

numero de variables = 24


#### Restriciones

**Numero de plazas disponibles**
$\sum_{i} \sum_{j} \sum_{k} X_{ijk} \leq 640$

**35% son premium**
$\sum_{j} \sum_{k} X_{2,j,k} \leq 640*0.35=224$

**Ocupacion 1/4 de estudiantes**
$\sum_{i} \sum_{j} X_{i,j,3} \leq \frac{1}{4}640=160$

**se venden un 40%  de premium**
$\sum_{j} \sum_{k} X_{2,j,k} \geq 224*40\%=112$

**Estudiantes adquieren 80% antes  de los 3 meses de los premium**
$\sum_{j} X_{2,j,3} \geq 112*80\% =89.6$

**Los adultos mayores ocupan como mínimo el 60% de la clase normal**
$\sum_{k} X_{1,k,2} \geq ((640*65\%)*60\%)=249.6$

**al menos la mitad de estas son adquiridas antes del último mes**
$X_{1,3,2} \geq 249.6*50\%=124.8$


In [6]:
model_data["num_restricciones"] = 7
model_data["Nombres restricciones"] = [
    "Numero de plazas disponibles",
    "0.35 son premium",
    "Ocupacion 1/4 de estudiantes",
    "Se venden un 0.4 de premium",
    "Estudiantes adquieren 0.8 antes  de los 3 meses de los premium",
    "Los adultos mayores ocupan como mínimo el 0.6 de la clase normal",
    "Al menos la mitad de estas son adquiridas antes del último mes",
]

model_data["coeficientes_restricciones"] = [
    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
    [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
]

model_data["valores_lado_derecho"] = [640, 224, 160, 112, 89.6, 249.6, 124.8]

In [7]:
for i in range(model_data["num_restricciones"]):
    restricciones = solver.RowConstraint(0, model_data["valores_lado_derecho"][i], model_data["Nombres restricciones"][i])

    for j in range(model_data["num_variables"]):
        restricciones.SetCoefficient(x[j],model_data["coeficientes_restricciones"][i][j])

print("Numero de restricciones =", solver.NumConstraints())

Numero de restricciones = 7


#### Funcion Objetivo

In [8]:
model_data["coeficientes_funcionOb"] = [
    60,
    56,
    50,
    52,
    90,
    88,
    82,
    86,
    # -
    72.5,
    68.5,
    62.5,
    64.5,
    108.75,
    106.75,
    100.75,
    104.75,
    # -
    90,
    86,
    80,
    82,
    135,
    135,
    135,
    135,
]

In [9]:
funcion_Objetivo = solver.Objective()

for j in range(model_data["num_variables"]):
    funcion_Objetivo.SetCoefficient(x[j], model_data["coeficientes_funcionOb"][j])

<h3 style="color:red">Verficar Max/Min</h3>

In [10]:
funcion_Objetivo.SetMaximization()

### Solucion

In [11]:
modelo=solver.Solve()

if modelo == pywraplp.Solver.OPTIMAL:
    print("Solución")
    print("Valor de la Función Objetivo a Maximizar: ", solver.Objective().Value())

    for j in range(model_data["num_variables"]):
        print(x[j].name(), " = ",x[j].solution_value())
else:
    print("El modelo no tiene una solución optima")

Solución
Valor de la Función Objetivo a Maximizar:  62640.0
Normal-6antes-persona  =  0.0
Normal-6antes-personaVieja  =  0.0
Normal-6antes-estudiante  =  0.0
Normal-6antes-joven  =  0.0
Premium-6antes-persona  =  0.0
Premium-6antes-personaVieja  =  0.0
Premium-6antes-estudiante  =  0.0
Premium-6antes-joven  =  0.0
Normal-3antes-persona  =  0.0
Normal-3antes-personaVieja  =  0.0
Normal-3antes-estudiante  =  0.0
Normal-3antes-joven  =  0.0
Premium-3antes-persona  =  0.0
Premium-3antes-personaVieja  =  0.0
Premium-3antes-estudiante  =  0.0
Premium-3antes-joven  =  0.0
Normal-normal-persona  =  528.0
Normal-normal-personaVieja  =  0.0
Normal-normal-estudiante  =  0.0
Normal-normal-joven  =  0.0
Premium-normal-persona  =  112.0
Premium-normal-personaVieja  =  0.0
Premium-normal-estudiante  =  0.0
Premium-normal-joven  =  0.0


## Cuarto Punto

### Proposición

La empresa Bavaria plantea su plan de producción para el año 2023, el cual le permita satisfacer la demanda de consumidores durante todo el año. La producción de cada litro de cerveza tiene un costo de 2 USD, este costo se mantiene durante el primer trimestre del año. Posterior a ello el costo incrementa a 3 USD durante el siguiente trimestre, y después de ello incrementa en un 75% para los meses siguientes. La empresa maneja unas demandas promedio de 120 mil, 120 mil, 140 mil litros de cerveza mensuales durante el primer trimestre, pero posterior a ello se incrementa la demanda 15 mil litros en relación con el mes que corresponde al trimestre anterior. En el último trimestre la demanda se duplica en relación con respecto al primer trimestre. La planta trabaja durante dos turnos al día, en el primer turno se operan 5 tanques de producción y en la tarde 4, cada tanque cuenta con capacidad de producción de 30 mil litros semanales de cerveza. El gerente indica que la operación durante el turno de la tarde incurre en costos adicionales de 25% por litro producido. En una situación de desabastecimiento la puede almacenar hasta 200 mil litros con un costo de 1,5 por litro almacenado. Definir el plan de producción que garantice satisfacer la demanda

### Modelo

In [ ]:
solver = pywraplp.Solver.CreateSolver("GLOP")
infinity = solver.infinity()

model_data = {}

#### Conjuntos

- $i$ = {Mes1, Mes2, Mes3, Mes4, Mes5, Mes6, Mes7, Mes8, Mes9, Mes10, Mes11, Mes12}
- $j$ = {Manana, Tarde}
- $k$ = {Mes1, Mes2, Mes3, Mes4, Mes5, Mes6, Mes7, Mes8, Mes9, Mes10, Mes11}


#### Variables

- $X_{ij} \rightarrow$ Cantidad de litros de Cerveza en el $M_i$ en $J_j$
- $Y_{ij} \rightarrow$ Cantidad de litros de Cerveza en el $M_k$ en $J_j$ vendidos en $V_{k+1}$ 

In [ ]:
model_data["num_variables"] = 48
model_data["Nombres variables"] = [
    "xMes1-Manana",
    "xMes2-Manana",
    "xMes3-Manana",
    "xMes4-Manana",
    "xMes5-Manana",
    "xMes6-Manana",
    "xMes7-Manana",
    "xMes8-Manana",
    "xMes9-Manana",
    "xMes10-Manana",
    "xMes11-Manana",
    "xMes12-Manana",
    "xMes1-Tarde",
    "xMes2-Tarde",
    "xMes3-Tarde",
    "xMes4-Tarde",
    "xMes5-Tarde",
    "xMes6-Tarde",
    "xMes7-Tarde",
    "xMes8-Tarde",
    "xMes9-Tarde",
    "xMes10-Tarde",
    "xMes11-Tarde",
    "xMes12-Tarde",
    # -
    "yMes1-Manana",
    "yMes2-Manana",
    "yMes3-Manana",
    "yMes4-Manana",
    "yMes5-Manana",
    "yMes6-Manana",
    "yMes7-Manana",
    "yMes8-Manana",
    "yMes9-Manana",
    "yMes10-Manana",
    "yMes11-Manana",
    "yMes1-Tarde",
    "yMes2-Tarde",
    "yMes3-Tarde",
    "yMes4-Tarde",
    "yMes5-Tarde",
    "yMes6-Tarde",
    "yMes7-Tarde",
    "yMes8-Tarde",
    "yMes9-Tarde",
    "yMes10-Tarde",
    "yMes11-Tarde",
]

In [ ]:
infinito = solver.infinity()
x = {}

for j in range(model_data["num_variables"]):
    x[j] = solver.Var(
        0, infinito, integer=False, name=model_data["Nombres variables"][j]
    )

print("numero de variables =", solver.NumVariables())

#### Restriciones

In [ ]:
model_data["num_restricciones"] = 15
model_data["Nombres restricciones"] = [
	"Capacidad tanque manana",
	"Capacidad tanque tarde",
    "Mes1"
    "Mes2"
    "Mes3"
    "Mes4"
    "Mes5"
    "Mes6"
    "Mes7"
    "Mes8"
    "Mes9"
    "Mes10"
    "Mes11"
    "Mes12"
    "Desbastecimiento"
	]

model_data["coeficientes_restricciones"] = [
    [],
]

model_data["valores_lado_derecho"] = [640, 224, 160, 112, 89.6, 249.6, 124.8]

In [ ]:
for i in range(model_data["num_restricciones"]):
    restricciones = solver.RowConstraint(0, model_data["valores_lado_derecho"][i], model_data["Nombres restricciones"][i])

    for j in range(model_data["num_variables"]):
        restricciones.SetCoefficient(x[j],model_data["coeficientes_restricciones"][i][j])

print("Numero de restricciones =", solver.NumConstraints())

#### Funcion Objetivo

In [ ]:
model_data["coeficientes_funcionOb"] = [
    60,
    56,
    50,
    52,
    90,
    88,
    82,
    86,
    # -
    72.5,
    68.5,
    62.5,
    64.5,
    108.75,
    106.75,
    100.75,
    104.75,
    # -
    90,
    86,
    80,
    82,
    135,
    135,
    135,
    135,
]

In [ ]:
funcion_Objetivo = solver.Objective()

for j in range(model_data["num_variables"]):
    funcion_Objetivo.SetCoefficient(x[j], model_data["coeficientes_funcionOb"][j])

<h3 style="color:red">Verficar Max/Min</h3>

In [ ]:

funcion_Objetivo.SetMaximization()

### Solucion